# 0.0 IMPORTS

In [1]:
import pandas as pd
import seaborn as sns
import umap.umap_ as umap
import numpy as np
import re
import sqlite3
from sqlalchemy import create_engine



import sweetviz
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm
from scipy.cluster import hierarchy as hc
from sklearn.decomposition import PCA
from sklearn import cluster as c
from sklearn import metrics as m
from sklearn import preprocessing as pp
from sklearn import ensemble as en
from sklearn.neighbors import NearestNeighbors
from sklearn import mixture as mx
from sklearn.manifold import TSNE
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

from IPython.display       import Image

c:\Users\Victor\Desktop\repos\pa005_clustering\clustering\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\Victor\Desktop\repos\pa005_clustering\clustering\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\Victor\Desktop\repos\pa005_clustering

## 0.2. Load Data

In [2]:
data_raw = pd.read_csv('datasets/Ecommerce.csv', encoding = 'unicode_escape')

data_raw = data_raw.drop(columns = ['Unnamed: 8'])

# 1.0. Data Description

In [3]:
df1 = data_raw.copy()

## 1.1. Rename Columns

In [4]:
df1.columns = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']

## 1.5. Replace NA

In [5]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [6]:
# create reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)

# merge original with reference dataframe
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

# coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop extra columns
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

## 1.6. Change Types

In [7]:
# invoice data
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer id
df1['customer_id'] = df1['customer_id'].astype('int64')

## 1.7. Descriptive Statistics

In [8]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [9]:
min_      = pd.DataFrame(num_attributes.apply(np.min)).T
max_      = pd.DataFrame(num_attributes.apply(np.max)).T
mean_     = pd.DataFrame(num_attributes.apply(np.mean)).T
median_   = pd.DataFrame(num_attributes.apply(np.median)).T
std_      = pd.DataFrame(num_attributes.apply(np.std)).T
range_    = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
skew_     = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
kurtosis_ = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenate
data_statistical = pd.concat([min_, max_, range_, mean_, median_, std_, skew_, kurtosis_]).T.reset_index()
data_statistical.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

In [10]:
data_statistical

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


#### 1.7.1.1 Numerical Attributes - Investigating

1. Negative Quantity (Devolution?)
2. Unit Price == 0 (Sales Promotion?)

### 1.7.2 Categorical Attributes

In [11]:
# INVOICE NUMBER

#cat_attributes.loc[cat_attributes['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))), 'invoice_no'].drop_duplicates()
len(cat_attributes.loc[cat_attributes['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))), 'invoice_no'].drop_duplicates())

3839

In [12]:
# STOCK CODE

#at_leat_one_string = len(cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('[^0-9]+', x))), 'stock_code'])
#at_leat_one_string

#cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()
df1.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), :].head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
45,536370,POST,POSTAGE,3,2016-11-29,18.00,France,12583
141,C536379,D,Discount,-1,2016-11-29,27.50,United Kingdom,14527
386,536403,POST,POSTAGE,1,2016-11-29,15.00,Netherlands,12791
1123,536527,POST,POSTAGE,1,2016-11-29,18.00,Germany,12662
1814,536544,DOT,DOTCOM POSTAGE,1,2016-11-29,569.77,United Kingdom,19001


# 2.0. Data Filtering

In [13]:
df2 = df1.copy()

In [14]:
# Numerical Attributes #
# Unit price > 0.0
df2 = df2.loc[df2['unit_price'] > 0.04, :]

# stock code != [POST, D, M, DOT, CRUK]
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'M', 'DOT', 'CRUK', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B'])]

# description
df2 = df2.drop(columns='description', axis=1)

# map
df2 = df2[~df2['country'].isin( [ 'European Community', 'Unspecified'] )]

# bad users
df2 = df2[~df2['customer_id'].isin([16466])]

## quantity - Negative numbers means product returns
df_returns = df2.loc[df2['quantity'] < 0, :]
df_purchase = df2.loc[df2['quantity'] > 0, :]

In [15]:
#df2.loc[df2['quantity'] < 0, :].sort_values(['customer_id', 'description']).head()
#df2.loc[df2['invoice_no'].apply(lambda x: bool(re.search('.540152', x))), :].head()
#df2.loc[df2['customer_id'] == 12395, :].sort_values('description')

# 3.0. Feature Engineering

In [16]:
df3 = df2.copy()

In [17]:
df3.head()

,invoice_no,stock_code,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,6,2016-11-29,3.39,United Kingdom,17850


## 3.1. Feature Creation

In [18]:
# data reference
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country'], axis = 1).drop_duplicates(ignore_index=True)

### 3.1.1 Gross Revenue

In [19]:
# Gross Revenue (Faturamento)
df_purchase.loc[:, 'gross_revenue'] = df_purchase.loc[:, 'quantity'] * df_purchase.loc[:, 'unit_price']

# Monetary
df_monetary = df_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

c:\Users\Victor\Desktop\repos\pa005_clustering\clustering\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
c:\Users\Victor\Desktop\repos\pa005_clustering\clustering\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


### 3.1.2 Recency - Day from last purchase

In [20]:
# Recency - Last day purchase
df_recency = df_purchase[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()

df_recency['recency_days'] = (df_purchase['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()

df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

### 3.1.3 Quantity of purchased

In [21]:
# Frequency
df_freq = (df_purchase[['customer_id', 'invoice_no']].drop_duplicates()
                                                    .groupby('customer_id')
                                                    .count()
                                                    .reset_index()
                                                    .rename(columns={'invoice_no':'qtde_invoices'}))


df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')

### 3.1.4 Quantity of items purchased

In [22]:
# Frequency
df_freq = (df_purchase[['customer_id', 'quantity']].groupby('customer_id')
                                                  .sum()
                                                  .reset_index()
                                                  .rename(columns={'quantity':'qtd_items'}))

df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')

### 3.1.5 Quantity of stock purchased

In [23]:
# Frequency
df_freq = (df_purchase[['customer_id', 'stock_code']].groupby('customer_id')
                                                  .count()
                                                  .reset_index()
                                                  .rename(columns={'stock_code':'qtd_products'}))

df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')

### 3.1.8 Frequency Purchase

In [24]:
df_aux = (df2[['invoice_no', 'customer_id', 'invoice_date']].drop_duplicates()
                                                            .groupby('customer_id')
                                                            .agg(max_ = ('invoice_date', 'max'),
                                                                 min_ = ('invoice_date', 'min'),
                                                                 days = ('invoice_date', lambda x: (( x.max() - x.min()).days) +1),
                                                                 buy_ = ('invoice_date', 'count'))).reset_index()

# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days']].apply(lambda x: x['buy_'] / x['days'] if x['days'] != 0 else 0, axis = 1)

df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')

### 3.1.9 Number of Returns

In [25]:
# Soma quantidade de devoluções
returns = ( df_returns[['customer_id', 'quantity']].groupby( 'customer_id' )
                                                       .sum()
                                                       .reset_index()
                                                       .rename( columns={'quantity': 'qtde_returns'} ) )

returns['qtde_returns'] = returns['qtde_returns'] * -1

df_ref = pd.merge( df_ref, returns, on='customer_id', how='left')
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0

df_ref.isna().sum()  

customer_id       0
gross_revenue    91
recency_days     91
qtde_invoices    91
qtd_items        91
qtd_products     91
frequency         0
qtde_returns      0
dtype: int64

In [26]:
df_ref.shape

(5786, 8)

# 4.0. EDA

In [27]:
df4 = df_ref.dropna().copy()

In [28]:
df4.shape

(5695, 8)

## 4.3. Estudo do Espaço

In [29]:
#df43 = df4.drop(columns=['customer_id'], axis=1).copy()
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtd_products', 'frequency', 'qtde_returns']
df43 = df4[cols_selected].copy()

In [30]:
mms = pp.MinMaxScaler()

In [31]:
df43['gross_revenue']          = mms.fit_transform(df43[['gross_revenue']])
df43['recency_days']           = mms.fit_transform(df43[['recency_days']])
df43['qtd_products']           = mms.fit_transform(df43[['qtd_products']])
df43['frequency']              = mms.fit_transform(df43[['frequency']])
df43['qtde_returns']           = mms.fit_transform(df43[['qtde_returns']])

### 4.3.4 Tree Based Embedding

In [32]:
X = df4.drop(columns=['customer_id', 'gross_revenue'], axis = 1)
y = df4['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor(n_estimators=100, random_state=42)

# model training
rf_model.fit(X, y)

# Leaf
df_leaf = pd.DataFrame(rf_model.apply(X))

In [33]:
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

c:\Users\Victor\Desktop\repos\pa005_clustering\clustering\lib\site-packages\numba\np\ufunc\parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


# 7.0. Hyperparameter Fine Tunning

In [34]:
#X = df6.drop(columns=['customer_id'])
#clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
X = df_tree.copy()
clusters = np.arange(2, 25, 1)

In [35]:
X.head()

,embedding_x,embedding_y
0,18.950182,11.863049
1,20.795233,9.913032
2,13.496415,18.219810
3,10.039892,-9.763520
4,5.008425,8.790209


# 8.0. Model Training

## 8.1 KMeans

In [36]:
# model difnition
k = 7
kmeans = c.KMeans(init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42)

# model training
kmeans.fit(X)

# clustering
labels = kmeans.labels_

### 8.1.1 Cluster Validation

In [37]:
## WSS
print(f'WSS value: {kmeans.inertia_}')

## SS (Silhouette Score)
print('SS value: {}'.format(m.silhouette_score(X, labels, metric='euclidean')))

WSS value: 116837.8828125
SS value: 0.4595277011394501


# 9.0. Cluster Analysis

In [38]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels
df92.head()

df92['recency_days'] = df92['recency_days'].astype('int64')
df92['qtd_products'] = df92['qtd_products'].astype('int64')
df92['qtde_returns'] = df92['qtde_returns'].astype('int64')

## 9.2. Cluster Profile

In [39]:
# Number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100*(df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# Avg Gross revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# Avg recency days
df_avg_recency_days = df92[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='left', on='cluster')

# qt products
df_qtde_products = df92[['qtd_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qtde_products, how='inner', on='cluster')

# Frequency
df_frequency = df92[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_frequency, how='inner', on='cluster')

# Qtde returns
df_qtde_returns = df92[['qtde_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qtde_returns, how='inner', on='cluster')

In [40]:
df_cluster.sort_values(by='gross_revenue', ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtd_products,frequency,qtde_returns
2,2,580,10.184372,8351.894379,103.265517,222.520690,0.541873,381.432759
3,3,627,11.009658,3253.113748,50.666667,213.641148,0.155032,23.711324
0,0,959,16.839333,1692.484922,70.175182,124.649635,0.264768,13.508863
6,6,1060,18.612818,960.536453,106.257547,75.497170,0.450271,5.128302
1,1,909,15.961370,477.253465,132.116612,46.655666,0.591222,2.614961
4,4,676,11.870061,265.535621,159.284024,16.553254,0.764165,2.028107
5,5,884,15.522388,150.220781,188.359729,12.464932,0.890429,0.940045


In [41]:
df92.dtypes

customer_id        int64
gross_revenue    float64
recency_days       int64
qtd_products       int64
frequency        float64
qtde_returns       int64
cluster            int32
dtype: object

# 11.0. Deploy to Production

In [43]:
df92.head()

,customer_id,gross_revenue,recency_days,qtd_products,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,0.486111,40,3
1,13047,3232.59,56,171,0.048780,35,3
2,12583,6705.38,2,232,0.045699,50,2
3,13748,948.25,95,28,0.017921,0,6
4,15100,876.00,333,3,0.136364,22,5


In [8]:
# create table
query_create_table_insiders =  """
    CREATE TABLE insiders (
        customer_id     INTEGER,
        gross_revenue   REAL,
        recency_days    INTEGER,
        qtd_products    INTEGER,
        frequency       REAL,
        qtde_returns    INTEGER,
        cluster         INTEGER
    )
"""

conn = sqlite3.connect('insiders_db.sqlite')
#conn.execute(query_create_table_insiders)
#conn.commit()
conn.close()

# insert data
#conn = create_engine('sqlite:///insiders_db.sqlite')
#df92.to_sql(name='insiders', con=conn, if_exists='append', index=False)



In [5]:
# consulting database
query = """
    SELECT * FROM insiders
"""

df = pd.read_sql_query(query, conn)

In [6]:
df

,customer_id,gross_revenue,recency_days,qtd_products,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,0.486111,40,3
1,13047,3232.59,56,171,0.048780,35,3
2,12583,6705.38,2,232,0.045699,50,2
3,13748,948.25,95,28,0.017921,0,6
4,15100,876.00,333,3,0.136364,22,5
...,...,...,...,...,...,...,...
11385,22706,5699.00,1,634,1.000000,0,3
11386,22707,6756.06,0,730,1.000000,0,3
11387,22708,3217.20,0,59,1.000000,0,0
11388,22709,3950.72,0,217,1.000000,0,0


AttributeError: 'Engine' object has no attribute 'close'